#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [11]:
#collection.delete_many({})

In [12]:
import pandas as pd
import pymongo

In [13]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [14]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [15]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

In [16]:
df_ks['goal']=df_ks['goal'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_ks['pledged']=df_ks['pledged'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_ks['backers']=df_ks['backers'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_ks['usd pledged']=df_ks['usd pledged'].apply(lambda x: pd.to_numeric(x, errors='coerce',downcast='float'))

In [17]:
df_ks.shape

(150000, 14)

## Question 0

### Netoyer les données

In [18]:
df_ks.dropna(axis=0,inplace=True)

In [19]:
df_sample=df_ks.sample(frac=1/4,axis=0)
df_sample.shape

(37129, 14)

In [20]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                float64
launched             object
pledged             float64
state                object
backers             float64
country              object
usd pledged         float64
usd_pledged_real    float64
dtype: object

In [21]:
df_ks['launched']=pd.to_datetime(df_ks['launched'])

In [15]:
df_ks['deadline']=pd.to_datetime(df_ks['deadline'])

In [16]:
df_ks.dtypes

ID                           int64
name                        object
category                    object
main_category               object
currency                    object
deadline            datetime64[ns]
goal                       float64
launched            datetime64[ns]
pledged                    float64
state                       object
backers                    float64
country                     object
usd pledged                float64
usd_pledged_real           float64
dtype: object

### Importer les données

In [17]:
def slice_generator(df, chunk_size=10):
    current_row = 0
    total_rows = df.shape[0]
    while current_row < total_rows:
        yield df[current_row:current_row + chunk_size]
        current_row += chunk_size


In [18]:
for df_chunk in slice_generator(df_sample):
    records = df_chunk.to_dict(orient='records')
    collection.insert_many(records)


In [19]:
collection.find_one()

{'_id': ObjectId('5ff2fbf352defb166e4e30a2'),
 'ID': 243679497,
 'name': "Lady Yum | Washington's First French Macaron Parlor",
 'category': 'Small Batch',
 'main_category': 'Food',
 'currency': 'USD',
 'deadline': '2014-05-30',
 'goal': 25000.0,
 'launched': '2014-05-01 22:14:13',
 'pledged': 37711.0,
 'state': 'successful',
 'backers': 333.0,
 'country': 'US',
 'usd pledged': 37711.0,
 'usd_pledged_real': 37711.0}

## Question 1  

In [20]:
cur = collection.aggregate([{"$group" : {"_id" : ["$ID","$name"], "averagebackers" : {"$avg" : "$backers"}}},{"$sort":{"averagebackers":-1}},{"$limit":5}])
print("les 5 projets ayant reçu le plus de promesse de dons sont:")
list(cur)

les 5 projets ayant reçu le plus de promesse de dons sont:


[{'_id': [286165030, 'Torment: Tides of Numenera'], 'averagebackers': 74405.0},
 {'_id': [1033978702, 'OUYA: A New Kind of Video Game Console'],
  'averagebackers': 63416.0},
 {'_id': [342886736,
   "COOLEST COOLER: 21st Century Cooler that's Actually Cooler"],
  'averagebackers': 62642.0},
 {'_id': [1957695328, 'Planetary Annihilation - A Next Generation RTS'],
  'averagebackers': 44162.0},
 {'_id': [474183945, 'BATTLETECH'], 'averagebackers': 41733.0}]

## Question 2

In [82]:
cur=collection.find({"state":"successful"})
n=len(list(cur))
print ("le nombre de projets ayant atteint leur but sont au nombre de :",n)


le nombre de projets ayant atteint leur but sont au nombre de : 13173


In [67]:
cur=collection.aggregate([{"$group" : {"_id" : "$state", "totalnumber" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'failed', 'totalnumber': 19744},
 {'_id': 'canceled', 'totalnumber': 3758},
 {'_id': 'successful', 'totalnumber': 13173},
 {'_id': 'suspended', 'totalnumber': 187},
 {'_id': 'live', 'totalnumber': 267}]

## Question 3

In [83]:
cur=collection.aggregate([{"$group":{"_id":"$category","total":{"$sum":1}}}])
print ("le nombre de projets pour chaque catégorie est :")
list(cur)

le nombre de projets pour chaque catégorie est :


[{'_id': 'Digital Art', 'total': 151},
 {'_id': 'Print', 'total': 71},
 {'_id': 'Animation', 'total': 247},
 {'_id': 'Puzzles', 'total': 17},
 {'_id': 'Gaming Hardware', 'total': 50},
 {'_id': 'Vegan', 'total': 51},
 {'_id': 'Community Gardens', 'total': 27},
 {'_id': 'Public Art', 'total': 329},
 {'_id': 'Immersive', 'total': 28},
 {'_id': 'Rock', 'total': 692},
 {'_id': 'Dance', 'total': 219},
 {'_id': 'Fiction', 'total': 932},
 {'_id': 'Movie Theaters', 'total': 24},
 {'_id': 'Comedy', 'total': 246},
 {'_id': 'Photobooks', 'total': 153},
 {'_id': 'Graphic Design', 'total': 187},
 {'_id': 'Games', 'total': 344},
 {'_id': 'Robots', 'total': 72},
 {'_id': 'Publishing', 'total': 574},
 {'_id': 'Punk', 'total': 28},
 {'_id': 'Zines', 'total': 43},
 {'_id': 'Documentary', 'total': 1628},
 {'_id': 'Quilts', 'total': 5},
 {'_id': 'Television', 'total': 98},
 {'_id': 'Makerspaces', 'total': 23},
 {'_id': 'Comic Books', 'total': 264},
 {'_id': 'Ceramics', 'total': 31},
 {'_id': 'Kids', 'total

## Question 4

In [90]:
cur=collection.find({"launched":{ "$lte" :  "2015-12-31"},"country":"FR"})
n=len(list(cur))
print("le nombre de projets français ayant été instanciés avant 2016 est : ",n)


le nombre de projets français ayant été instanciés avant 2016 est :  96


## Question 5

In [101]:
cur=collection.find({"pledged":{ "$gte" :  200000},"country":'US'})
print("Les projets américains ayant demandé plus de 200 000 dollars sont :")
list(cur)


Les projets américains ayant demandé plus de 200 000 dollars sont :


[{'_id': ObjectId('5fd3571c247348e355276857'),
  'ID': 1866482244,
  'name': 'Deep Learning for Computer Vision with Python [ eBook ]',
  'category': 'Technology',
  'main_category': 'Technology',
  'currency': 'USD',
  'deadline': '2017-02-17',
  'goal': 5000.0,
  'launched': '2017-01-18 15:59:32',
  'pledged': 262792.0,
  'state': 'successful',
  'backers': 1014.0,
  'country': 'US',
  'usd pledged': 84104.0,
  'usd_pledged_real': 262792.0},
 {'_id': ObjectId('5fd3571c247348e3552768c8'),
  'ID': 1288353288,
  'name': 'ORION4Sight: Sunglasses On Steroids',
  'category': 'Wearables',
  'main_category': 'Technology',
  'currency': 'USD',
  'deadline': '2014-09-13',
  'goal': 9000.0,
  'launched': '2014-08-12 18:37:49',
  'pledged': 348449.0,
  'state': 'successful',
  'backers': 2881.0,
  'country': 'US',
  'usd pledged': 348449.0,
  'usd_pledged_real': 348449.0},
 {'_id': ObjectId('5fd3571d247348e355276a7b'),
  'ID': 545788185,
  'name': "World's First And Only Complete Weather Camera 

In [102]:
cur=collection.find({"pledged":{ "$gte" :  200000},"country":"US"})
a=len(list(cur))
print("Le nombre de projets américains ayant demandé plus de 200 000 dollars est : ", a)


Le nombre de projets américains ayant demandé plus de 200 000 dollars est :  158


## Question 6 

In [103]:
cur=collection.find({"name" : {"$regex": ".*Sport.*"}})
n=len(list(cur))
print("Le nombre de projet ayant 'Sport' dans leur nom : ", n)

Le nombre de projet ayant 'Sport' dans leur nom :  79
